In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras.optimizers import Adam, Nadam
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model
from sklearn.metrics import roc_auc_score

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
MAX_WORDS = 20000
MAX_NB_CHARS = 1000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

## Data

In [4]:
data = pd.read_csv('../input/articles.csv')

In [5]:
data = data[['text', 'claps']]

In [6]:
data['claps'] = data['claps'].apply(lambda s: int(float(s[:-1]) * 1000) if s[-1] == 'K' else int(s))

In [7]:
text = data['text']

In [8]:
sentences = text.apply(lambda x: x.split())

In [9]:
tokenizer = Tokenizer(num_words=MAX_NB_CHARS, char_level=True)
tokenizer.fit_on_texts(sentences.values)

In [10]:
data_text = np.zeros((len(sentences), MAX_WORDS), dtype='int32')
for i, words in enumerate(sentences):
    k = 0
    for j, word in enumerate(words):
        if j < MAX_WORDS:
            if tokenizer.word_index[word] < MAX_NB_CHARS:
                data_text[i, k] = tokenizer.word_index[word]
                k=k+1

In [11]:
char_index = tokenizer.word_index
print('Total %s unique tokens.' % len(char_index))

Total 41227 unique tokens.


In [12]:
claps = data['claps'].values
claps = claps.reshape(claps.shape[0], 1)
print('Shape of data tensor:', data_text.shape)
print('Shape of clap tensor:', claps.shape)

Shape of data tensor: (337, 20000)
Shape of clap tensor: (337, 1)


In [13]:
indices = np.arange(data_text.shape[0])
np.random.shuffle(indices)
data_text = data_text[indices]
claps = claps[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data_text.shape[0])

x_train = data_text[:-nb_validation_samples]
y_train = claps[:-nb_validation_samples]
x_val = data_text[-nb_validation_samples:]
y_val = claps[-nb_validation_samples:]

## Model design

### Attention Layer

In [14]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [15]:
def get_embedding(char_index):
    return Embedding(len(char_index) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_WORDS,
                            trainable=True)

def create_model(embedding=None):
    inp = Input(shape=(MAX_WORDS,))
    x = embedding(inp)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(1, activation="relu")(x)

    model = Model(inputs=inp, outputs=x)

    model.compile(loss='mse', optimizer=Adam(lr=1e-3, clipnorm=4), metrics=['accuracy'])

    return model

In [16]:
embedding = get_embedding(char_index)

In [17]:
model = create_model(embedding)

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20000)             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20000, 300)        12368400  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20000, 128)        186880    
_________________________________________________________________
attention_with_context_1 (At (None, 128)               16640     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 12,572,049
Trainable params: 12,572,049
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 32
epochs = 10
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_val, y_val))